Loading dataset and see if we can retireve information from it.

In [2]:
import pandas as pd

# Load the CSV file to inspect the data
file_path = 'domain_properties.csv'
data = pd.read_csv(file_path)

# Display the first few rows and basic information about the dataset
data.head(), data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11160 entries, 0 to 11159
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   price                     11160 non-null  int64  
 1   date_sold                 11160 non-null  object 
 2   suburb                    11160 non-null  object 
 3   num_bath                  11160 non-null  int64  
 4   num_bed                   11160 non-null  int64  
 5   num_parking               11160 non-null  int64  
 6   property_size             11160 non-null  int64  
 7   type                      11160 non-null  object 
 8   suburb_population         11160 non-null  int64  
 9   suburb_median_income      11160 non-null  int64  
 10  suburb_sqkm               11160 non-null  float64
 11  suburb_lat                11160 non-null  float64
 12  suburb_lng                11160 non-null  float64
 13  suburb_elevation          11160 non-null  int64  
 14  cash_r

(    price date_sold         suburb  num_bath  num_bed  num_parking  \
 0  530000   13/1/16      Kincumber         4        4            2   
 1  525000   13/1/16     Halekulani         2        4            2   
 2  480000   13/1/16  Chittaway Bay         2        4            2   
 3  452000   13/1/16        Leumeah         1        3            1   
 4  365500   13/1/16    North Avoca         0        0            0   
 
    property_size         type  suburb_population  suburb_median_income  \
 0           1351        House               7093                 29432   
 1            594        House               2538                 24752   
 2            468        House               2028                 31668   
 3            344        House               9835                 32292   
 4           1850  Vacant land               2200                 45084   
 
    suburb_sqkm  suburb_lat  suburb_lng  suburb_elevation  cash_rate  \
 0        9.914   -33.47252   151.40208         

Concerting date_sold into a datetime object and selecting the features needed for VAR analysis.

In [3]:
# Convert the 'date_sold' column to datetime format and set it as the index
data['date_sold'] = pd.to_datetime(data['date_sold'], format='%d/%m/%y')
data.set_index('date_sold', inplace=True)
# Establish date_sold as our index
# inplace=True means we modify the original dataframe


# Select relevant columns for VAR analysis
var_data = data[['price', 'num_bath', 'num_bed', 'num_parking', 'property_size', 
                 'suburb_population', 'suburb_median_income', 'cash_rate', 'property_inflation_index']]



# Display the prepared data
var_data.head()


,price,num_bath,num_bed,num_parking,property_size,suburb_population,suburb_median_income,cash_rate,property_inflation_index
date_sold,,,,,,,,,
2016-01-13,530000,4,4,2,1351,7093,29432,2.0,150.9
2016-01-13,525000,2,4,2,594,2538,24752,2.0,150.9
2016-01-13,480000,2,4,2,468,2028,31668,2.0,150.9
2016-01-13,452000,1,3,1,344,9835,32292,2.0,150.9
2016-01-13,365500,0,0,0,1850,2200,45084,2.0,150.9


Creating the VAR model
* We need to check if the time series is stationary.
A stationary series has statistical properties (such as mean and variance) that do not change over time. This assumption is crucial because it allows the model to capture the relationships between variables consistently over time. If the data is not stationary, the model might produce misleading results.

Non-stationary data can lead to inaccurate forecasts. Stationary data helps in building more reliable and accurate forecasts because the model can better understand the underlying patterns and relationships between variables.

The ADF test is used to check if a time series is stationary. It tests the null hypothesis that a unit root is present in the time series data (i.e., the series is non-stationary). A p-value below a certain threshold (commonly 0.05) indicates that the null hypothesis can be rejected, suggesting that the series is stationary.

In [4]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.api import VAR

# Function to perform the Augmented Dickey-Fuller test for stationarity
def adf_test(series):
    result = adfuller(series, autolag='AIC')
    return result[1]  # Return the p-value

# Check for stationarity and difference the data if necessary
stationarity_results = var_data.apply(adf_test)
non_stationary_columns = stationarity_results[stationarity_results > 0.05].index

# If any columns are non-stationary, difference them
if len(non_stationary_columns) > 0: # Do we have columns that have trends?
    var_data_diff = var_data.diff().dropna() 
else:
    var_data_diff = var_data


# var_data.diff(): Computes the first difference of each column in the DataFrame.
# This subtracts each value from the previous value, effectively removing trends.

# Non-stationary time series often exhibit trends or systematic changes over time. 
# Differencing can help remove these trends. By focusing on the change between periods,
# rather than the absolute values, the resulting series may exhibit a constant mean 
# and variance over time.

# Trends add complexity that the basic VAR model is not designed to handle.
# If a VAR model includes trends, the forecasts generated may be unreliable or biased. 

# Fit the VAR model
model = VAR(var_data_diff)
lag_order = model.select_order(maxlags=15)
selected_lag = lag_order.selected_orders['aic']

# Fit the model with the selected lag
var_model_fitted = model.fit(selected_lag)

# Summary of the VAR model
var_model_fitted.summary()


c:\Users\erikh\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sat, 10, Aug, 2024
Time:                     19:58:07
--------------------------------------------------------------------
No. of Equations:         9.00000    BIC:                    71.6657
Nobs:                     11144.0    HQIC:                   71.1325
Log likelihood:          -535932.    FPE:                5.95554e+30
AIC:                      70.8619    Det(Omega_mle):     5.33960e+30
--------------------------------------------------------------------
Results for equation price
                                  coefficient       std. error           t-stat            prob
-----------------------------------------------------------------------------------------------
const                              715.739265     11315.712902            0.063           0.950
L1.price                            -0.616762         0.011043          -55.849           0.000


In [5]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Split the data into training and testing sets (e.g., last 20% of the data as test)
train_size = int(len(var_data_diff) * 0.8)
train, test = var_data_diff[:train_size], var_data_diff[train_size:]

# Fit the VAR model on the training set
model = VAR(train)
var_model_fitted = model.fit(selected_lag)

# Generate forecasts for the test set period
forecast = var_model_fitted.forecast(train.values, steps=len(test))

# Convert forecast to a DataFrame for easier comparison
forecast_df = pd.DataFrame(forecast, index=test.index, columns=test.columns)

# Calculate the performance metrics
mae = mean_absolute_error(test, forecast_df)
rmse = np.sqrt(mean_squared_error(test, forecast_df))
mape = np.mean(np.abs((test - forecast_df) / test)) * 100
r2 = r2_score(test, forecast_df)

# Display the metrics
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared: {r2}")


c:\Users\erikh\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE): 30181.797252884407
Root Mean Squared Error (RMSE): 365804.2575091268
R-squared: 3.1552676550565074e-05


Mean Absolute Error (MAE):

Purpose: Measures the average magnitude of errors in a set of predictions, without considering their direction. Lower MAE indicates better predictive accuracy. 

Root Mean Squared Error (RMSE):

Purpose: Measures the square root of the average of the squared differences between predicted and actual values.
RMSE penalizes larger errors and is sensitive to outliers and shows how the model performs on average.

R-squared 

Purpose: R-squared ranges from 0 to 1, with higher values indicating a better fit.


R^2 might not adequately reflect model performance. For instance, in models that account for lagged effects (like ARIMA or VAR), R^2 might not tell the whole story about how well the model is capturing temporal dependencies.

In [7]:
df = pd.DataFrame(data)

# Calculate Mean
mean_value = df['price'].mean()

# Calculate Median
median_value = df['price'].median()

# Calculate Q1 (25th percentile)
Q1 = df['price'].quantile(0.25)

# Calculate Q3 (75th percentile)
Q3 = df['price'].quantile(0.75)

# Calculate Interquartile Range (IQR)
IQR = Q3 - Q1

# Output the results
print("Mean:", mean_value)
print("Median:", median_value)
print("Q1 (25th percentile):", Q1)
print("Q3 (75th percentile):", Q3)
print("IQR (Interquartile Range):", IQR)

# Find the lowest value
lowest_value = df['price'].min()

# Find the highest value
highest_value = df['price'].max()

# Output the results
print("Lowest Value:", lowest_value)
print("Highest Value:", highest_value)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared: {r2}")

Mean: 1675395.2672939068
Median: 1388000.0
Q1 (25th percentile): 1002000.0
Q3 (75th percentile): 2020000.0
IQR (Interquartile Range): 1018000.0
Lowest Value: 225000
Highest Value: 60000000
Mean Absolute Error (MAE): 30181.797252884407
Root Mean Squared Error (RMSE): 365804.2575091268
R-squared: 3.1552676550565074e-05


Compared to CAT, this is a much more better performing model as its RMSE is $365,804. However, this needs to be reduced to be more accurate and reliable.